In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cluster import KMeans

In [2]:
nba_data = pd.read_csv('Data/nba_data.csv')
players = nba_data.player.to_list()


In [3]:
nba_data = nba_data.set_index('rank')
nba_data.drop(['Unnamed: 0', 'player', 'pos', 'team_id', 'age', 'mp', 'fg', 'fga','fg3',
 'fg3a','fg2',
 'fg2a','ft',
 'fta','orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',], axis=1, inplace=True)
list(nba_data.columns.values)


['g',
 'gs',
 'fg_pct',
 'fg3_pct',
 'fg2_pct',
 'efg_pct',
 'ft_pct',
 'mp_per_g',
 'fg_per_g',
 'fga_per_g',
 'fg3_per_g',
 'fg3a_per_g',
 'fg2_per_g',
 'fg2a_per_g',
 'ft_per_g',
 'fta_per_g',
 'orb_per_g',
 'drb_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'tov_per_g',
 'pf_per_g',
 'pts_per_g',
 'fg_per_mp',
 'fga_per_mp',
 'fg3_per_mp',
 'fg3a_per_mp',
 'fg2_per_mp',
 'fg2a_per_mp',
 'ft_per_mp',
 'fta_per_mp',
 'orb_per_mp',
 'drb_per_mp',
 'trb_per_mp',
 'ast_per_mp',
 'stl_per_mp',
 'blk_per_mp',
 'tov_per_mp',
 'pf_per_mp',
 'pts_per_mp',
 'fg_per_poss',
 'fga_per_poss',
 'fg3_per_poss',
 'fg3a_per_poss',
 'fg2_per_poss',
 'fg2a_per_poss',
 'ft_per_poss',
 'fta_per_poss',
 'orb_per_poss',
 'drb_per_poss',
 'trb_per_poss',
 'ast_per_poss',
 'stl_per_poss',
 'blk_per_poss',
 'tov_per_poss',
 'pf_per_poss',
 'pts_per_poss',
 'off_rtg',
 'def_rtg',
 'per',
 'ts_pct',
 'fg3a_per_fga_pct',
 'fta_per_fga_pct',
 'orb_pct',
 'drb_pct',
 'trb_pct',
 'ast_pct',
 'stl

In [4]:
nba_data
offprofile_df = nba_data[['fg_pct', 'fg3_pct', 'fg2_pct', 'efg_pct', 'ft_pct','ts_pct', 'fg3a_per_fga_pct', 
                           'fta_per_fga_pct', 'fg_per_poss', 'fga_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 
                          'fg2_per_poss', 'fg2a_per_poss', 'ft_per_poss', 'fta_per_poss', 'pts_per_poss', 
                          'usg_pct', 'ast_pct', 'tov_pct', 'orb_pct']]

In [5]:
nba_data_minus_games = nba_data.drop(['g',
 'gs'], axis=1)

nba_data_minus_games

,fg_pct,fg3_pct,fg2_pct,efg_pct,ft_pct,mp_per_g,fg_per_g,fga_per_g,fg3_per_g,fg3a_per_g,...,tov_pct,usg_pct,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp
rank,,,,,,,,,,,,,,,,,,,,,
1,0.357,0.323,0.500,0.487,0.923,19.0,1.8,5.1,1.3,4.1,...,7.9,12.2,0.1,0.6,0.6,0.053,-2.4,-0.9,-3.4,-0.2
3,0.345,0.338,0.361,0.459,0.778,12.6,1.1,3.2,0.7,2.2,...,19.7,13.5,-0.1,0.2,0.1,0.011,-3.1,-1.3,-4.4,-0.3
4,0.595,0.000,0.596,0.595,0.500,33.4,6.0,10.1,0.0,0.0,...,12.6,16.4,5.1,4.0,9.1,0.163,0.6,2.1,2.7,3.2
5,0.576,0.200,0.588,0.579,0.735,23.3,3.4,5.9,0.0,0.2,...,17.1,15.8,3.4,3.4,6.8,0.171,-0.6,3.6,3.0,2.4
8,0.519,0.238,0.528,0.522,0.847,33.2,8.4,16.3,0.1,0.5,...,8.8,26.9,6.4,2.9,9.3,0.167,1.2,0.5,1.6,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,0.422,0.341,0.515,0.513,0.748,27.2,2.8,6.6,1.2,3.5,...,11.5,13.2,0.3,2.0,2.2,0.052,-1.7,0.6,-1.2,0.4
488,0.559,0.292,0.586,0.572,0.795,22.3,6.1,11.0,0.3,1.0,...,12.7,28.5,2.2,1.9,4.2,0.183,0.1,1.2,1.2,0.9
521,0.521,0.346,0.571,0.560,0.732,12.0,2.9,5.6,0.4,1.2,...,10.7,26.1,0.4,0.3,0.8,0.151,-1.5,-1.8,-3.3,-0.1


In [6]:
nba_data_minus_advanced = nba_data.drop(['per',
 'ts_pct',
 'fg3a_per_fga_pct',
 'fta_per_fga_pct',
 'orb_pct',
 'drb_pct',
 'trb_pct',
 'ast_pct',
 'stl_pct',
 'blk_pct',
 'tov_pct',
 'usg_pct',
 'ows',
 'dws',
 'ws',
 'ws_per_48',
 'obpm',
 'dbpm',
 'bpm',
 'vorp',
 'off_rtg',
 'def_rtg'], axis=1)

In [7]:
nba_advanced_data = nba_data.drop(['g',
 'gs',
    'fg_pct',
 'fg3_pct',
 'fg2_pct',
 'efg_pct',
 'ft_pct',
 'mp_per_g',
 'fg_per_g',
 'fga_per_g',
 'fg3_per_g',
 'fg3a_per_g',
 'fg2_per_g',
 'fg2a_per_g',
 'ft_per_g',
 'fta_per_g',
 'orb_per_g',
 'drb_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'tov_per_g',
 'pf_per_g',
 'pts_per_g',
 'fg_per_mp',
 'fga_per_mp',
 'fg3_per_mp',
 'fg3a_per_mp',
 'fg2_per_mp',
 'fg2a_per_mp',
 'ft_per_mp',
 'fta_per_mp',
 'orb_per_mp',
 'drb_per_mp',
 'trb_per_mp',
 'ast_per_mp',
 'stl_per_mp',
 'blk_per_mp',
 'tov_per_mp',
 'pf_per_mp',
 'pts_per_mp',
 'fg_per_poss',
 'fga_per_poss',
 'fg3_per_poss',
 'fg3a_per_poss',
 'fg2_per_poss',
 'fg2a_per_poss',
 'ft_per_poss',
 'fta_per_poss',
 'orb_per_poss',
 'drb_per_poss',
 'trb_per_poss',
 'ast_per_poss',
 'stl_per_poss',
 'blk_per_poss',
 'tov_per_poss',
 'pf_per_poss',
 'pts_per_poss',], axis=1)

In [8]:
pd.options.display.max_rows = 100

def run_cluster(data, num_clusters=5):
    # create kmeans model
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(data)

    # find centers and labels
    centers = kmeans.cluster_centers_
    labels = kmeans.labels_

    # DataFrame of centroids
    centroids_df = pd.DataFrame(centers, columns=data.columns).T

    # print items in clusters
    clusters = {}
    for i in range(len(labels)):
        clusters[players[i]] = labels[i]

    for i in range(len(centers)):
        out = [k for k,v in clusters.items() if v==i]

        print('-'*30)
        print(f'cluster index: {i}')
        for item in out:
            print(item)
        print('\n')

    centroids_df.to_csv('test3.csv')
    return centroids_df

In [9]:
run_cluster(offprofile_df, 7)

------------------------------
cluster index: 0
LaMarcus Aldridge
Deandre Ayton
Marvin Bagley
Thomas Bryant
Clint Capela
John Collins
Gorgui Dieng
Andre Drummond
Derrick Favors
Harry Giles
Rudy Gobert
Montrezl Harrell
Willy Hernangomez
Rondae Hollis-Jefferson
Richaun Holmes
Dwight Howard
Serge Ibaka
Jaren Jackson
T.J. Leaf
Alex Len
Robin Lopez
JaVale McGee
Paul Millsap
Jusuf Nurkic
Jahlil Okafor
Domantas Sabonis
Hassan Whiteside
Ante Zizic
Kenneth Faried
Enes Kanter
Boban Marjanovic
Greg Monroe
Bobby Portis
Jonas Valanciunas
Christian Wood
Ivica Zubac


------------------------------
cluster index: 1
Michael Beasley
Antonio Blakeney
Bogdan Bogdanovic
Bojan Bogdanovic
Malcolm Brogdon
MarShon Brooks
Jaylen Brown
Jordan Clarkson
DeMarcus Cousins
Spencer Dinwiddie
Tyreke Evans
Evan Fournier
Danilo Gallinari
Rudy Gay
Paul George
Aaron Gordon
Buddy Hield
Aaron Holiday
Brandon Ingram
Josh Jackson
Reggie Jackson
Frank Kaminsky
Kyle Kuzma
Jeremy Lamb
Kawhi Leonard
Caris LeVert
Kevin Love
Trey L

,0,1,2,3,4,5,6
fg_pct,0.552139,0.449436,0.421758,0.545151,0.423562,0.464438,0.429029
fg3_pct,0.217000,0.353818,0.337846,0.167453,0.358888,0.335625,0.322956
fg2_pct,0.574389,0.498000,0.509868,0.571811,0.486764,0.514938,0.476735
efg_pct,0.566694,0.512782,0.511363,0.560377,0.514618,0.517938,0.488529
ft_pct,0.701556,0.802127,0.729967,0.666491,0.788202,0.794687,0.755324
ts_pct,0.596444,0.553182,0.535824,0.586792,0.544551,0.564656,0.520676
fg3a_per_fga_pct,0.091500,0.354236,0.522560,0.103755,0.505820,0.307656,0.353750
fta_per_fga_pct,0.360694,0.261618,0.187736,0.366057,0.196820,0.319438,0.224191
fg_per_poss,10.186111,9.798182,5.323077,6.828302,7.270787,11.846875,6.685294
fga_per_poss,18.586111,21.809091,12.616484,12.660377,17.221348,25.546875,15.550000


In [10]:
run_cluster(nba_data_minus_games, 7)

------------------------------
cluster index: 0
J.J. Barea
Bradley Beal
Eric Bledsoe
Devin Booker
Mike Conley
Stephen Curry
DeMar DeRozan
Luka Doncic
Kevin Durant
De'Aaron Fox
Paul George
Blake Griffin
James Harden
Jrue Holiday
Kyrie Irving
LeBron James
Zach LaVine
Kawhi Leonard
Damian Lillard
Donovan Mitchell
Victor Oladipo
Chris Paul
Derrick Rose
D'Angelo Russell
Ben Simmons
Kemba Walker
John Wall
Russell Westbrook
Lou Williams
Trae Young


------------------------------
cluster index: 1
Al-Farouq Aminu
Kyle Anderson
OG Anunoby
Ryan Arcidiacono
Dwayne Bacon
Nicolas Batum
Malik Beasley
Marco Belinelli
Davis Bertans
Patrick Beverley
Nemanja Bjelica
Jaron Blossomgame
Jonah Bolden
Mikal Bridges
Miles Bridges
Ryan Broekhoff
Sterling Brown
Bruno Caboclo
Kentavious Caldwell-Pope
DeMarre Carroll
Vince Carter
Gary Clark
Pat Connaughton
Quinn Cook
Allen Crabbe
Torrey Craig
Jae Crowder
Dante Cunningham
Seth Curry
Luol Deng
Damyean Dotson
Jared Dudley
Terrance Ferguson
Yogi Ferrell
Dorian Finney

,0,1,2,3,4,5,6
fg_pct,0.458700,0.440709,0.446954,0.43122,0.541478,0.393128,0.553412
fg3_pct,0.334700,0.359682,0.356169,0.33918,0.251174,0.304654,0.180544
fg2_pct,0.511100,0.517727,0.495323,0.48100,0.569870,0.461077,0.579265
efg_pct,0.518867,0.535864,0.512754,0.49334,0.562130,0.463962,0.570706
ft_pct,0.798533,0.766918,0.783554,0.75808,0.738174,0.733987,0.686544
mp_per_g,33.006667,21.600909,28.581538,22.97800,29.247826,14.920513,17.336765
fg_per_g,7.940000,2.810000,5.507692,3.27200,7.091304,1.869231,2.808824
fga_per_g,17.336667,6.397273,12.343077,7.59800,13.239130,4.733333,5.097059
fg3_per_g,2.150000,1.234545,1.660000,0.95000,0.595652,0.674359,0.188235
fg3a_per_g,6.020000,3.352727,4.553846,2.74800,1.869565,2.152564,0.576471


In [11]:
run_cluster(nba_data_minus_advanced, 7)

------------------------------
cluster index: 0
Steven Adams
Jarrett Allen
Al-Farouq Aminu
D.J. Augustin
Nicolas Batum
Nemanja Bjelica
Eric Bledsoe
Bojan Bogdanovic
Malcolm Brogdon
Clint Capela
Willie Cauley-Stein
Darren Collison
Derrick Favors
Terrance Ferguson
Bryn Forbes
Evan Fournier
De'Aaron Fox
Shai Gilgeous-Alexander
Rudy Gobert
Aaron Gordon
Jerami Grant
Danny Green
Draymond Green
Joe Harris
Al Horford
Joe Ingles
Jonathan Isaac
Reggie Jackson
Brook Lopez
Kyle Lowry
JaVale McGee
Paul Millsap
Cedi Osman
J.J. Redick
Josh Richardson
Ricky Rubio
Collin Sexton
Pascal Siakam
Ben Simmons
Jayson Tatum
P.J. Tucker
Myles Turner
Marvin Williams
Thaddeus Young
Trevor Ariza
Harrison Barnes
Marc Gasol
Justin Holiday
DeAndre Jordan
Wesley Matthews


------------------------------
cluster index: 1
Justin Anderson
Dwayne Bacon
Mo Bamba
Aron Baynes
Jordan Bell
Khem Birch
Antonio Blakeney
Jonah Bolden
Ryan Broekhoff
Sterling Brown
Troy Brown
Jose Calderon
Gary Clark
Ian Clark
Pat Connaughton
Dante 

,0,1,2,3,4,5,6
g,76.52000,52.674419,73.172414,70.595238,27.830769,48.52000,72.704545
gs,74.56000,5.372093,10.839080,49.476190,3.861538,32.86000,71.954545
fg_pct,0.48094,0.452523,0.458724,0.454262,0.430277,0.45494,0.475795
fg3_pct,0.32538,0.278372,0.322414,0.339405,0.302862,0.29584,0.330477
fg2_pct,0.52998,0.512721,0.517356,0.517048,0.486785,0.50086,0.522432
efg_pct,0.54788,0.511733,0.528736,0.524762,0.494185,0.51030,0.530750
ft_pct,0.76924,0.721047,0.758368,0.773905,0.721108,0.73464,0.807000
mp_per_g,29.93600,14.453488,21.333333,25.728571,14.940000,25.50400,33.311364
fg_per_g,4.85800,1.952326,3.364368,3.873810,2.016923,4.29400,8.109091
fga_per_g,10.16400,4.382558,7.421839,8.571429,4.693846,9.53600,17.127273


In [12]:
run_cluster(nba_advanced_data, 7)

------------------------------
cluster index: 0
Al-Farouq Aminu
Kyle Anderson
Marvin Bagley
Mo Bamba
Aron Baynes
Jordan Bell
Bismack Biyombo
Nemanja Bjelica
Jonah Bolden
Bruno Caboclo
Wendell Carter
Omri Casspi
Zach Collins
Dewayne Dedmon
Cheick Diallo
Gorgui Dieng
Cristiano Felicio
Rudy Gay
Harry Giles
Aaron Gordon
Marcin Gortat
Isaiah Hartenstein
Nene Hilario
Dwight Howard
Serge Ibaka
Ersan Ilyasova
Jonathan Isaac
Jaren Jackson
Jonas Jerebko
Amir Johnson
Derrick Jones
Frank Kaminsky
Maxi Kleber
Kosta Koufos
T.J. Leaf
Alex Len
Jon Leuer
Ian Mahinmi
Lauri Markkanen
Jarell Martin
Alfonzo McKinnie
Salah Mejri
Paul Millsap
Marcus Morris
Jahlil Okafor
Kelly Olynyk
Zaza Pachulia
Alex Poythress
Ivan Rabb
Thabo Sefolosha
Omari Spellman
Jayson Tatum
Tristan Thompson
Myles Turner
Noah Vonleh
Johnathan Williams
D.J. Wilson
Guerschon Yabusele
Thaddeus Young
Ante Zizic
Tyson Chandler
Henry Ellenson
JaMychal Green
Tobias Harris
Thon Maker
Nikola Mirotic
Greg Monroe
Jabari Parker
Bobby Portis
Dario 

,0,1,2,3,4,5,6
off_rtg,112.166667,107.681159,97.853933,116.705882,112.234234,129.347826,112.923077
def_rtg,109.055556,111.536232,112.303371,106.352941,112.243243,106.956522,110.461538
per,14.700000,13.566667,9.088764,21.852941,12.218919,19.530435,21.130769
ts_pct,0.568306,0.528710,0.492775,0.596059,0.563108,0.649391,0.566692
fg3a_per_fga_pct,0.262569,0.369971,0.434820,0.137088,0.502766,0.089609,0.349462
fta_per_fga_pct,0.296361,0.232000,0.196292,0.374059,0.196099,0.405174,0.316269
orb_pct,7.858333,2.243478,2.835955,9.782353,3.109910,11.239130,3.023077
drb_pct,20.491667,10.853623,12.637079,26.635294,12.410811,20.804348,14.861538
trb_pct,14.179167,6.555072,7.697753,18.220588,7.752252,16.026087,8.961538
ast_pct,8.818056,23.978261,10.752809,15.652941,9.457658,8.560870,34.150000
